<a href="https://colab.research.google.com/github/mmontielpz/premier-data-scout/blob/main/noteboks/data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Required Packages

In [36]:
!pip install soccerdata

# Load Required Libraries

In [37]:
import soccerdata as sd
import pandas as pd
import numpy as np

# Load & Standardize EPL Salaries Dataset

In [38]:
# Load EPL Salaries dataset
epl_salaries = pd.read_csv("/content/soccer_salaries.csv")

# Standardize column names
epl_salaries.columns = epl_salaries.columns.str.strip().str.lower()

# Rename 'player name' column to 'player' for merging
epl_salaries.rename(columns={'player name': 'player'}, inplace=True)

# Function to clean currency values
def clean_currency(value):
    if isinstance(value, str):
        return float(value.replace("£", "").replace(",", "").strip())  # Remove currency symbols & commas
    return value  # If already numeric, return as is

# Apply cleaning function to salary columns
epl_salaries["weekly wage"] = epl_salaries["weekly wage"].apply(clean_currency)
epl_salaries["yearly salary"] = epl_salaries["yearly salary"].apply(clean_currency)

print("✅ EPL Salaries Dataset Cleaned & Standardized")
print(epl_salaries.head())


✅ EPL Salaries Dataset Cleaned & Standardized
   unnamed: 0           player  weekly wage  yearly salary  age   position  \
0           0             Neto     100000.0      5200000.0   33         GK   
1           1  Dominic Solanke      72000.0      3744000.0   25         ST   
2           2  Justin Kluivert      70000.0      3640000.0   24      AM RL   
3           3   Philip Billing      54000.0      2808000.0   27   DM, AM C   
4           4       Alex Scott      53000.0      2756000.0   19  DM, AM RC   

   nationality             team  
0       Brazil  afc-bournemouth  
1      England  afc-bournemouth  
2  Netherlands  afc-bournemouth  
3      Denmark  afc-bournemouth  
4      England  afc-bournemouth  


In [39]:
epl_salaries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 630 entries, 0 to 629
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   unnamed: 0     630 non-null    int64  
 1   player         630 non-null    object 
 2   weekly wage    630 non-null    float64
 3   yearly salary  630 non-null    float64
 4   age            630 non-null    int64  
 5   position       630 non-null    object 
 6   nationality    630 non-null    object 
 7   team           630 non-null    object 
dtypes: float64(2), int64(2), object(4)
memory usage: 39.5+ KB


# Fetch Player Stats from FBref

In [40]:
# Initialize FBref for the English Premier League (2024 season)
fbref = sd.FBref(leagues="ENG-Premier League", seasons=2024)

# Get player stats for attackers (goal & shot creation)
player_creation_stats = fbref.read_player_season_stats(stat_type="goal_shot_creation")

# Get player stats for defenders (defensive actions)
player_def_stats = fbref.read_player_season_stats(stat_type="defense")

# Get miscellaneous stats (includes player names)
player_misc_stats = fbref.read_player_season_stats(stat_type="misc")

[03/14/25 15:34:21] INFO     Saving cached data to /root/soccerdata/data/FBref                       ]8;id=275870;file:///usr/local/lib/python3.11/dist-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=758471;file:///usr/local/lib/python3.11/dist-packages/soccerdata/_common.py#263\263]8;;\

[03/14/25 15:34:22] WARNING  /usr/local/lib/python3.11/dist-packages/soccerdata/fbref.py:165:       ]8;id=459506;file:///usr/lib/python3.11/warnings.py\warnings.py]8;;\:]8;id=365730;file:///usr/lib/python3.11/warnings.py#110\110]8;;\
                             FutureWarning: The behavior of DataFrame concatenation with empty or                  
                             all-NA entries is deprecated. In a future version, this will no longer                
                             exclude empty or all-NA columns when determining the result dtypes. To                
                             retain the old behavior, exclude the relevant entries before the                      
                             concat operation.                                                                     
                               pd.concat(dfs)                                                                      
                                                                                                                   

                    WARNING  /usr/local/lib/python3.11/dist-packages/soccerdata/fbref.py:165:       ]8;id=909109;file:///usr/lib/python3.11/warnings.py\warnings.py]8;;\:]8;id=86153;file:///usr/lib/python3.11/warnings.py#110\110]8;;\
                             FutureWarning: The behavior of DataFrame concatenation with empty or                  
                             all-NA entries is deprecated. In a future version, this will no longer                
                             exclude empty or all-NA columns when determining the result dtypes. To                
                             retain the old behavior, exclude the relevant entries before the                      
                             concat operation.                                                                     
                               pd.concat(dfs)                                                                      
                                                                                                                   

[03/14/25 15:34:23] WARNING  /usr/local/lib/python3.11/dist-packages/soccerdata/fbref.py:165:       ]8;id=786393;file:///usr/lib/python3.11/warnings.py\warnings.py]8;;\:]8;id=350361;file:///usr/lib/python3.11/warnings.py#110\110]8;;\
                             FutureWarning: The behavior of DataFrame concatenation with empty or                  
                             all-NA entries is deprecated. In a future version, this will no longer                
                             exclude empty or all-NA columns when determining the result dtypes. To                
                             retain the old behavior, exclude the relevant entries before the                      
                             concat operation.                                                                     
                               pd.concat(dfs)                                                                      
                                                                                                                   

# Clean & Standarize Data

In [41]:
# Flatten MultiIndex columns (if necessary)
for df in [player_creation_stats, player_def_stats, player_misc_stats]:
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = ["_".join(col).strip() if col[1] else col[0] for col in df.columns]

# Convert 'age' to numeric, forcing non-numeric values to NaN
for df in [player_creation_stats, player_def_stats]:
    df['age'] = pd.to_numeric(df['age'], errors='coerce')

# Compute median safely (only if there are valid values)
for df in [player_creation_stats, player_def_stats]:
    median_age = df['age'].median()  # Compute median
    if pd.isna(median_age):  # If the median is NaN, use a default value
        median_age = 25  # You can change this default value as needed

    df['age'].fillna(int(median_age), inplace=True)  # Fill NaN with median
    df['age'] = df['age'].astype(int)  # Convert final values to integer

                    WARNING  /usr/local/lib/python3.11/dist-packages/numpy/lib/nanfunctions.py:1215 ]8;id=765127;file:///usr/lib/python3.11/warnings.py\warnings.py]8;;\:]8;id=703128;file:///usr/lib/python3.11/warnings.py#110\110]8;;\
                             : RuntimeWarning: Mean of empty slice                                                 
                               return np.nanmean(a, axis, out=out, keepdims=keepdims)                              
                                                                                                                   

                    WARNING  <ipython-input-41-7188a4f3dba1>:16: FutureWarning: A value is trying   ]8;id=146029;file:///usr/lib/python3.11/warnings.py\warnings.py]8;;\:]8;id=545090;file:///usr/lib/python3.11/warnings.py#110\110]8;;\
                             to be set on a copy of a DataFrame or Series through chained                          
                             assignment using an inplace method.                                                   
                             The behavior will change in pandas 3.0. This inplace method will never                
                             work because the intermediate object on which we are setting values                   
                             always behaves as a copy.                                                             
                                                                                                                   
                             For example, when doing 'df.method(value, inplace=True)', try using                   
                             'df.method({col: value}, inplace=True)' or df = df.method(value)                      
                             instead, to perform the operation inplace on the original object.                     
                                                                                                                   
                                                                                                                   
                               df['age'].fillna(int(median_age), inplace=True)  # Fill NaN with                    
                             median                                                                                
                                                                                                                   

[03/14/25 15:34:24] WARNING  /usr/local/lib/python3.11/dist-packages/numpy/lib/nanfunctions.py:1215 ]8;id=279345;file:///usr/lib/python3.11/warnings.py\warnings.py]8;;\:]8;id=71182;file:///usr/lib/python3.11/warnings.py#110\110]8;;\
                             : RuntimeWarning: Mean of empty slice                                                 
                               return np.nanmean(a, axis, out=out, keepdims=keepdims)                              
                                                                                                                   

                    WARNING  <ipython-input-41-7188a4f3dba1>:16: FutureWarning: A value is trying   ]8;id=532229;file:///usr/lib/python3.11/warnings.py\warnings.py]8;;\:]8;id=147894;file:///usr/lib/python3.11/warnings.py#110\110]8;;\
                             to be set on a copy of a DataFrame or Series through chained                          
                             assignment using an inplace method.                                                   
                             The behavior will change in pandas 3.0. This inplace method will never                
                             work because the intermediate object on which we are setting values                   
                             always behaves as a copy.                                                             
                                                                                                                   
                             For example, when doing 'df.method(value, inplace=True)', try using                   
                             'df.method({col: value}, inplace=True)' or df = df.method(value)                      
                             instead, to perform the operation inplace on the original object.                     
                                                                                                                   
                                                                                                                   
                               df['age'].fillna(int(median_age), inplace=True)  # Fill NaN with                    
                             median                                                                                
                                                                                                                   

# Ensure player Column Exists & Reset Index

In [42]:
# Reset multi-index if necessary
for df in [player_misc_stats, player_creation_stats, player_def_stats]:
    df.reset_index(inplace=True)

# Fix player column issue
for df in [player_creation_stats, player_def_stats]:
    if 'player_x' in df.columns:
        df.rename(columns={'player_x': 'player'}, inplace=True)
    elif 'player_y' in df.columns:
        df.rename(columns={'player_y': 'player'}, inplace=True)

# Check available columns
print("Columns in player_creation_stats:", player_creation_stats.columns.tolist())
print("Columns in player_def_stats:", player_def_stats.columns.tolist())


Columns in player_creation_stats: ['league', 'season', 'team', 'player', 'nation', 'pos', 'age', 'born', '90s', 'SCA_SCA', 'SCA_SCA90', 'SCA Types_PassLive', 'SCA Types_PassDead', 'SCA Types_TO', 'SCA Types_Sh', 'SCA Types_Fld', 'SCA Types_Def', 'GCA_GCA', 'GCA_GCA90', 'GCA Types_PassLive', 'GCA Types_PassDead', 'GCA Types_TO', 'GCA Types_Sh', 'GCA Types_Fld', 'GCA Types_Def']
Columns in player_def_stats: ['league', 'season', 'team', 'player', 'nation', 'pos', 'age', 'born', '90s', 'Tackles_Tkl', 'Tackles_TklW', 'Tackles_Def 3rd', 'Tackles_Mid 3rd', 'Tackles_Att 3rd', 'Challenges_Tkl', 'Challenges_Att', 'Challenges_Tkl%', 'Challenges_Lost', 'Blocks_Blocks', 'Blocks_Sh', 'Blocks_Pass', 'Int', 'Tkl+Int', 'Clr', 'Err']


# Merge Player Names into Attackers & Defenders

In [43]:
# Ensure 'player' exists in player_misc_stats
if 'player' not in player_misc_stats.columns:
    player_misc_stats.reset_index(inplace=True)

# Merge player names into attackers & defenders datasets
player_creation_stats = player_creation_stats.merge(
    player_misc_stats[['player']], how="left", left_index=True, right_index=True
)

player_def_stats = player_def_stats.merge(
    player_misc_stats[['player']], how="left", left_index=True, right_index=True
)

print("✅ Player names merged successfully.")


✅ Player names merged successfully.


In [44]:
# Ensure only one 'player' column is kept
for df in [player_creation_stats, player_def_stats]:
    if 'player_x' in df.columns and 'player_y' in df.columns:
        df['player'] = df['player_x'].combine_first(df['player_y'])  # Keep non-null values
        df.drop(columns=['player_x', 'player_y'], inplace=True)  # Drop extra columns
    elif 'player_x' in df.columns:
        df.rename(columns={'player_x': 'player'}, inplace=True)
    elif 'player_y' in df.columns:
        df.rename(columns={'player_y': 'player'}, inplace=True)

# Check if player column is now correct
print("✅ Fixed player column. Available columns:", player_creation_stats.columns.tolist())


✅ Fixed player column. Available columns: ['league', 'season', 'team', 'nation', 'pos', 'age', 'born', '90s', 'SCA_SCA', 'SCA_SCA90', 'SCA Types_PassLive', 'SCA Types_PassDead', 'SCA Types_TO', 'SCA Types_Sh', 'SCA Types_Fld', 'SCA Types_Def', 'GCA_GCA', 'GCA_GCA90', 'GCA Types_PassLive', 'GCA Types_PassDead', 'GCA Types_TO', 'GCA Types_Sh', 'GCA Types_Fld', 'GCA Types_Def', 'player']


In [45]:
player_creation_stats.head()

,league,season,team,nation,pos,age,born,90s,SCA_SCA,SCA_SCA90,...,SCA Types_Def,GCA_GCA,GCA_GCA90,GCA Types_PassLive,GCA Types_PassDead,GCA Types_TO,GCA Types_Sh,GCA Types_Fld,GCA Types_Def,player
0,ENG-Premier League,2425,Arsenal,ENG,DF,25,1997,7.8,9,1.15,...,0,1,0.13,1,0,0,0,0,0,Ben White
1,ENG-Premier League,2425,Arsenal,ENG,"FW,MF",25,2001,14.1,89,6.32,...,0,21,1.49,12,5,2,1,1,0,Bukayo Saka
2,ENG-Premier League,2425,Arsenal,ESP,GK,25,1995,28.0,3,0.11,...,0,0,0.0,0,0,0,0,0,0,David Raya
3,ENG-Premier League,2425,Arsenal,ENG,MF,25,1999,23.0,84,3.66,...,2,7,0.3,4,2,0,1,0,0,Declan Rice
4,ENG-Premier League,2425,Arsenal,ENG,"FW,MF",25,2007,6.6,16,2.42,...,0,2,0.3,2,0,0,0,0,0,Ethan Nwaneri


# Merge Salaries

In [46]:
# Rename 'player name' to 'player' in EPL salaries dataset
epl_salaries.rename(columns={'player name': 'player'}, inplace=True)

# Merge salaries with player stats (Attackers & Defenders)
player_creation_stats = player_creation_stats.merge(
    epl_salaries[['player', 'weekly wage', 'yearly salary']], how='left', on='player'
)

player_def_stats = player_def_stats.merge(
    epl_salaries[['player', 'weekly wage', 'yearly salary']], how='left', on='player'
)

# Fill missing salary values with NaN (it will be NaN by default in Pandas)
print("✅ Salaries merged successfully.")

✅ Salaries merged successfully.


In [47]:
player_creation_stats.head()

,league,season,team,nation,pos,age,born,90s,SCA_SCA,SCA_SCA90,...,GCA_GCA90,GCA Types_PassLive,GCA Types_PassDead,GCA Types_TO,GCA Types_Sh,GCA Types_Fld,GCA Types_Def,player,weekly wage,yearly salary
0,ENG-Premier League,2425,Arsenal,ENG,DF,25,1997,7.8,9,1.15,...,0.13,1,0,0,0,0,0,Ben White,NaN,NaN
1,ENG-Premier League,2425,Arsenal,ENG,"FW,MF",25,2001,14.1,89,6.32,...,1.49,12,5,2,1,1,0,Bukayo Saka,250000.0,13000000.0
2,ENG-Premier League,2425,Arsenal,ESP,GK,25,1995,28.0,3,0.11,...,0.0,0,0,0,0,0,0,David Raya,77000.0,4004000.0
3,ENG-Premier League,2425,Arsenal,ENG,MF,25,1999,23.0,84,3.66,...,0.3,4,2,0,1,0,0,Declan Rice,250000.0,13000000.0
4,ENG-Premier League,2425,Arsenal,ENG,"FW,MF",25,2007,6.6,16,2.42,...,0.3,2,0,0,0,0,0,Ethan Nwaneri,NaN,NaN


# Filter & Save Attackers and Defenders

In [48]:
# Filter only forwards (attackers)
attackers = player_creation_stats[player_creation_stats["pos"].str.contains("FW", na=False)]

# Filter only defenders
defenders = player_def_stats[player_def_stats["pos"].str.contains("DF", na=False)]

# Save both datasets
attackers.to_csv("/content/cleaned_attackers.csv", index=False)
defenders.to_csv("/content/cleaned_defenders.csv", index=False)

print("✅ Cleaned datasets saved: 'cleaned_attackers.csv' and 'cleaned_defenders.csv'")

✅ Cleaned datasets saved: 'cleaned_attackers.csv' and 'cleaned_defenders.csv'


In [49]:
!pip install nbconvert[webpdf] playwright pyppeteer

In [50]:
!playwright install

Playwright Host validation warning: 
╔══════════════════════════════════════════════════════╗
║ Host system is missing dependencies to run browsers. ║
║ Missing libraries:                                   ║
║     libgtk-4.so.1                                    ║
║     libgraphene-1.0.so.0                             ║
║     libwoff2dec.so.1.0.2                             ║
║     libgstgl-1.0.so.0                                ║
║     libgstcodecparsers-1.0.so.0                      ║
║     libavif.so.13                                    ║
║     libharfbuzz-icu.so.0                             ║
║     libenchant-2.so.2                                ║
║     libsecret-1.so.0                                 ║
║     libhyphen.so.0                                   ║
║     libmanette-0.2.so.0                              ║
╚══════════════════════════════════════════════════════╝
    at validateDependenciesLinux (/usr/local/lib/python3.11/dist-packages/playwright/driver/package/lib/serv

In [51]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [52]:
!jupyter nbconvert --to webpdf /content/drive/My\Drive/proyecto/data-cleaning.ipynb --output "data-cleaning.pdf"

[NbConvertApp] Converting notebook /content/drive/MyDrive/proyecto/data-cleaning.ipynb to webpdf
[NbConvertApp] Building PDF
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 191929 bytes to /content/drive/MyDrive/proyecto/data-cleaning.pdf.pdf
